In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [2]:
#reads in the csv file
matches = pd.read_csv("matchesPrem(2024-2019).csv", index_col = 0)

#shows the type of each data entry
matches.dtypes

#function declaration to decide W, D, or L
def result(WLD):
    if WLD == 'W':
        return 3
    elif WLD == 'D':
        return 1
    else:
        return 0



In [3]:
#ML Base(Data Adjustment and Cleaning)


#adjusts the type of date index
matches["date"] = pd.to_datetime(matches["date"])

#sets home/away to a numeric value
matches["venue_code"] = matches["venue"].astype("category").cat.codes

#creates column with opponent as a number
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

#creates an hour column for time of day
matches["hour"] = matches["time"].str.replace(":.+", "", regex = True).astype("int")

#gets day of week as number
matches["day_code"] = matches["date"].dt.dayofweek

#sets numeric value for wins losses and draws
matches["target"] = matches["result"].apply(result)

#sets wins to 1 and losses and draws to 0
# matches["target"] = (matches["result"]=='W').astype("int")



In [4]:
#Implementing ML Model

#creates the random forst trees to be trained
rf = RandomForestClassifier(n_estimators = 50, min_samples_split = 10, random_state = 1)

#segregates data by training or testing
train = matches[matches["date"] < '2024-06-20']
# print(train.size)
test = matches[matches["date"] > '2024-06-20']
# print(test.size)


#sets predictors(what it uses to predict)
predictors = ["venue_code","opp_code","hour","day_code"]

#fits the RF model
rf.fit(train[predictors], train["target"])

#predicts
preds = rf.predict(test[predictors])

In [5]:
#seeing results

#scores accuracy
acc = accuracy_score(test["target"], preds)
combined = pd.DataFrame(dict(actual = test["target"], prediction = preds))
pd.crosstab(index = combined["actual"], columns = combined["prediction"])

#accuracy of predicting a win
precision_score(test["target"], preds, average = "micro")
# precision_score(test["target"], preds)

0.4371069182389937

In [6]:
#implementing rolling averages

#groups matches together by team
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City")

#function to get the average of last 3 matches
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed = 'left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset = new_cols)
    return group

#groups to be calculated tolling averages for
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"rolling_{c}" for c in cols]
# print(matches_rolling,predictors+new_cols)

#applies rolling to every match
matches_rolling = matches.groupby("team").apply(lambda x:rolling_averages(x, cols, new_cols))

#drops the extra extra column for team name
matches_rolling = matches_rolling.droplevel("team")

#makes every match its own index rather than start over for each team
matches_rolling.index = range(matches_rolling.shape[0])


In [ ]:
#Predictions 2.0 for W, D/L

def make_predictions(data, predictors):
    train = data[data["date"] < '2024-06-20']
    test = data[data["date"] > '2024-06-20']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual = test["target"], predicted = preds), index = test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

combined, precision = make_predictions(matches_rolling, predictors + new_cols)

combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index = True, right_index = True)
precision

In [7]:
#Predictions 3.0 for W, L, D

def make_predictions(data, predictors):
    train = data[data["date"] < '2024-06-20']
    test = data[data["date"] > '2024-06-20']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual = test["target"], predicted = preds), index = test.index)
    precision = precision_score(test["target"], preds, average = "micro")
    return combined, precision

combined, precision = make_predictions(matches_rolling, predictors + new_cols)

combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index = True, right_index = True)
precision

0.45396825396825397

In [8]:
#combining both sides of a match
class MissingDict(dict):
    __missing__ = lambda self, key: key
    
map_vals = {
    "Brighton and Hove Albion": "Brighton", 
    "Manchester United": "Manchester Utd", 
    "Newcastle United": "Newcastle Utd", 
    "Tottenham Hotspur": "Tottenham", 
    "West Ham United": "West Ham", 
    "Wolverhampton Wanderers": "Wolves"
}
mapping = MissingDict(**map_vals)

combined["new_team"] = combined["team"].map(mapping)

merged = combined.merge(combined, left_on = ["date", "new_team"], right_on= ["date", "opponent"])
merged
#merged[(merged["predicted_x"] == 3) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()
#merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 1)].value_counts()
#merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()
#merged[(merged["predicted_x"] == 0) & (merged["predicted_y"] == 3)]["actual_x"].value_counts()
merge_df = merged

merge_df.to_csv("test.csv")